In [1]:
import igraph as ig
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import powerlaw

from tqdm import trange, tqdm

from collections import defaultdict
from itertools import combinations

from load_graph import *

In [8]:
def measure_params(g, coms, min_degree=5, min_com_size=10):
    """
    Measure parameters of a graph with known communities.

    input:
    - g: the graph (igraph). Each vertex must have it's a set of it's communities stored in a 'comms' attribute.
    - coms: a list of lists containing the vertex id's in each community
    - min_degree  5: override the minimum degree parameter if the actual min degree is too small.
        This value will also be used for computeing the degree powerlaw exponent.
    - min_com_size  10: override the minimum community size parameter if the actual min com size is too small.
        This value will also be used for computeing the community size powerlaw exponent.

    returns:
    - params: a dictionary with the measured parameters
    """
    params = dict()

    n_coms = np.array([len(comms) for comms in g.vs["comms"]])
    is_outlier = n_coms == 0
    params["n"] = int(g.vcount())
    params["n_out"] = int(np.sum(is_outlier))

    params["eta"] = float(np.mean(n_coms))

    degrees = np.array(g.degree())
    d_min = np.maximum(min_degree, np.min(degrees))
    params["d_min"] = d_min
    params["d_max"] = int(np.max(degrees))
    params["t1"] = powerlaw.Fit(degrees, discrete=True, verbose=False, xmin=d_min).power_law.alpha

    com_sizes = np.array([len(com) for com in coms])
    c_min = np.maximum(min_com_size, np.min(com_sizes))
    params["c_min"] = c_min
    params["c_max"] = int(np.max(com_sizes))
    params["t2"] = powerlaw.Fit(com_sizes, discrete=True, verbose=False, xmin=c_min).power_law.alpha

    xi = sum([len(g.vs[e.source]["comms"].intersection(g.vs[e.target]["comms"]))==0 for e in g.es]) / g.ecount()
    params["xi"] = xi

    rho = np.corrcoef(degrees, [len(c) for c in g.vs["comms"]])[0, 1]
    params["rho"] = rho

    return params

In [9]:
# # Read Graph
graph_path = "data/com-youtube.ungraph.txt"
com_path = "data/com-youtube.all.cmty.txt"
drop_outliers=True

# graph_path = "data/com-dblp.ungraph.txt"
# com_path = "data/com-dblp.all.cmty.txt"
# drop_outliers=False

# graph_path = "data/com-amazon.ungraph.txt"
# com_path = "data/com-amazon.all.dedup.cmty.txt"
# drop_outliers=False

g, coms = load_snap(graph_path, com_path, drop_outliers)
measure_params(g, coms)

{'n': 52675,
 'n_out': 0,
 'eta': 2.4528144280968203,
 'd_min': 5,
 'd_max': 1928,
 't1': 1.8702187087097446,
 'c_min': 10,
 'c_max': 3001,
 't2': 2.130965769664415,
 'xi': 0.5928066048845747,
 'rho': 0.3746343169285614}